In [ ]:
!pip install datasets

## Load Dataset From Huggingface

In [ ]:
import datasets as dts

dataset = dts.load_dataset('squad_v1_pt')
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

## Normalize Dataset

- Use the format proposed by [lmqg/qg_squad](https://huggingface.co/datasets/lmqg/qg_squad)

- Replace HTML codecs for real strings



In [ ]:
dataset['train']['context'][:3]

['Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro do edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços erguidos com a lenda &quot;Venite Ad Me Omnes&quot;. Ao lado do edifício principal é a Basílica do Sagrado Coração. Imediatamente atrás da basílica é a Gruta, um lugar mariano de oração e reflexão. É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858. No final da unidade principal (e em uma linha direta que liga através de 3 estátuas e da Cúpula de Ouro), é um estátua de pedra simples e moderna de Maria.',
 'Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro do edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços er

In [ ]:
from html.entities import name2codepoint
import re

REGEX_PATTERN = re.compile(r'(?<=\&)\w+(?=\;)')

def replace_htmlcodecs(text):

    for match_obj in re.finditer(REGEX_PATTERN, text):

        str_pattern = match_obj.group()
        text = re.sub(f'\&{str_pattern}\;', chr(name2codepoint[str_pattern]), text)

    return text


In [ ]:
def preprocess_dataset(example):

    question = replace_htmlcodecs(example["question"])
    context = replace_htmlcodecs(example['context'])
    example['paragraph'] = context
    example["question"] =  question
    example["answer"] =  replace_htmlcodecs(example["answers"]["text"][0])
    example["paragraph_id"] =  example["id"]
    example["paragraph_question"] = f"question: {question} paragraph: {context}"

    return example

In [ ]:
processed_dataset = dataset.map(preprocess_dataset,
                                remove_columns=['answers', 'title', 'context','id'])


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
from google.colab import userdata

processed_dataset.push_to_hub('qg_squad_v1_pt', token=userdata.get('huggingface'))

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/88 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/525 [00:00<?, ?B/s]

In [ ]:
context = processed_dataset['train']['paragraph'][0]
question =  processed_dataset['train']['question'][0]
answer =  processed_dataset['train']['answer'][0]
paragraph_question =  processed_dataset['train']['paragraph_question'][0]

print("Context: ", context)
print("Question: ", question)
print("Answer: ", answer)
print("paragraph_question: ", paragraph_question)

Context:  Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro do edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços erguidos com a lenda "Venite Ad Me Omnes". Ao lado do edifício principal é a Basílica do Sagrado Coração. Imediatamente atrás da basílica é a Gruta, um lugar mariano de oração e reflexão. É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858. No final da unidade principal (e em uma linha direta que liga através de 3 estátuas e da Cúpula de Ouro), é um estátua de pedra simples e moderna de Maria.
Question:  A quem a Virgem Maria supostamente apareceu em 1858 em Lourdes, na França?
Answer:  Saint Bernadette Soubirous
paragraph_question:  question: A quem a Virgem Maria supostamente apareceu em 1858 em Lourdes, na França? paragraph: Arquitetonicamente, a esc

## Extract extra information from the dataset using Spacy (In Progress)


- **Sentence**: Sentence where the answer is found.

- **Sentence Answer:** Sentence where the answer is found with the answer highlighted with `<h1> <h1>` tags.

- **Paragraph Answer**: The paragraph with the answer highlighted with `<h1> <h1>` tags.

- **Paragraph Sentence**: The paragraph with the sentence highlighted with `<h1> <h1>` tags.

### Group Paragraphs based on context text.

In [ ]:
text_tuples = []

def get_question_answers_dict(question, answer):
    return {"question": replace_htmlcodecs(question),
            "answer": replace_htmlcodecs(answer)}


for i, group in dataset['train'].to_pandas().groupby('context'):

    context = replace_htmlcodecs(group['context'].tolist()[0])
    questions = group['question'].tolist()
    answers = group['answers'].tolist()
    question_answers = [get_question_answers_dict(questions[j], answer["text"][0],)
                                              for j,answer in enumerate(answers)
               if len(answer["text"]) > 0]
    text_tuples.append((
        context, question_answers
    ))



In [ ]:
text_tuples[:2]

[('"4 Minutes" foi lançado como o single principal do álbum e alcançou o terceiro lugar na Billboard Hot 100. Foi o top 10 das paradas de Madonna no chart - empurrou Madonna para além de Elvis Presley como o artista com os dez maiores sucessos. . No Reino Unido, ela manteve o seu recorde de mais singles número um para uma artista feminina; "4 minutos", tornando-se seu décimo terceiro. No 23º Prêmio do Disco de Ouro do Japão, Madonna recebeu seu quinto troféu de Artista do Ano da Associação da Indústria Fonográfica do Japão, o mais importante para qualquer artista. Para promover ainda mais o álbum, Madonna embarcou no Sticky & Sweet Tour; seu primeiro grande empreendimento com a Live Nation. Com uma bruta de US $ 280 milhões, tornou-se a turnê de maior bilheteria de uma artista solo, superando o recorde anterior de Madonna com a Confessions Tour; foi mais tarde superado por The Wall Live, de Roger Waters. Ele foi estendido para o ano seguinte, adicionando novas datas européias, e depois

### Spacy setup

- Functions and components definitions

- Attributed definitions


In [ ]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Token, Doc
from spacy.language import Language

if not Doc.has_extension("questions"):
    Doc.set_extension("questions", default=[])

if not Doc.has_extension("answers"):
    Doc.set_extension("answers", default=[])

if not Doc.has_extension("answer_sentence"):
    Doc.set_extension("answer_sentence", default=[])


def get_sentence(token):
    """
     Gets the sentence where the answer is found.
    """

    for sent in token.doc.sents:
        if sent.start <= token.i <= sent.end:
            return sent


def spacy_mark_text_span(text_doc, start_span, end_span,):
  context_words = [token.text for token in text_doc]

  spaces = [not token.is_punct for token in text_doc]
  spaces.pop(0)
  spaces += [False]
  #FIX: punctuations are one index after where it should be
  spaces.insert(start_span,False)
  spaces.insert(end_span,False)

  context_words.insert(start_span, "<h1>")
  context_words.insert(end_span+1, "<h1>")

  return Doc(nlp.vocab, words=context_words, spaces=spaces)

@Language.component("context_component")
def context_component(doc):
    """
    Adds the questions and answers as doc attributes
    """
    for qa_pair in doc._context:

      doc._.questions.append(qa_pair["question"])
      doc._.answers.append(nlp.make_doc(qa_pair["answer"]))

    return doc


@Language.component("paragraph_answer")
def paragraph_answer(doc):
    """
    Find the answer inside the paragragh using Ruled Based Matcher
    """
    matcher = Matcher(nlp.vocab)

    for i, answer in enumerate(doc._.answers):
        pattern = [[{"ORTH": token.text} ]  for token in answer]
        matcher.add(answer.text, pattern)
        matches = matcher(doc)

        start_sent, end_sent = -1, -1

        answers_starts = []
        answers_ends = []
        for match_id, start, end in matches:
            doc._.answer_sentence.append(doc[start]._.sent)

        doc._.paragraph_sentence.append(spacy_mark_text_span(doc, min(answers_starts), max(answers_ends)))
    return doc


# Add a computed property, which will be accessible as token._.sent
Token.set_extension('sent', getter=get_sentence, force=True)


nlp = spacy.blank('pt')
nlp.add_pipe('sentencizer')
# nlp.add_pipe("context_component", name="context_component")
# nlp.add_pipe("answer_component", name="answer_component")


In [ ]:
from spacy.lang.pt import Portuguese

nlp('casa').similarity('casinha')

<ipython-input-42-02bf2c9ed442>:3: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  nlp('casa').similarity('casinha')
<ipython-input-42-02bf2c9ed442>:3: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  nlp('casa').similarity('casinha')


0.0

In [ ]:
from spacy.symbols import IS_DIGIT, IS_PUNCT, ORTH
#, {IS_PUNCT: True}, {IS_DIGIT: True}
special_case = [{ORTH: "24"},{ORTH: "–"}, {ORTH: "10"}]
nlp.tokenizer.add_special_case("24–10", special_case)

In [ ]:

doc_tuples = nlp.pipe(text_tuples, as_tuples=True)



In [ ]:
for doc, info in doc_tuples:
  print(doc._.paragraph_answer[0])
  print(doc._.answers[0])
  print(doc._.questions[0])
  break

1 72
1 72
1 78
1 195
1 208
" <h1>4 Minutes" foi lançado como o single principal do álbum e alcançou o terceiro lugar na Billboard Hot 100. Foi o top 10 das paradas de Madonna no chart- empurrou Madonna para além de Elvis Presley como o artista com os dez maiores sucessos.. No Reino Unido, ela manteve o seu recorde de mais singles número um para uma artista feminina;" 4 minutos<h1>", tornando-se seu décimo terceiro. No 23º Prêmio do Disco de Ouro do Japão, Madonna recebeu seu quinto troféu de Artista do Ano da Associação da Indústria Fonográfica do Japão, o mais importante para qualquer artista. Para promover ainda mais o álbum, Madonna embarcou no Sticky& Sweet Tour; seu primeiro grande empreendimento com a Live Nation. Com uma bruta de US $ 280 milhões, tornou-se a turnê de maior bilheteria de uma artista solo, superando o recorde anterior de Madonna com a Confessions Tour; foi mais tarde superado por The Wall Live, de Roger Waters. Ele foi estendido para o ano seguinte, adicionando n

In [ ]:
matcher = Matcher(nlp.vocab)
ahead_pattern = [ {"ORTH": '&'}, {"TEXT": {"REGEX": '\w+'}}]
matcher.add('htmlcodec', [ahead_pattern, ])

## Format Dataset


Remove html condecs patterns and get the answers as a string instead of dictionary. Moreover, it changes the names of the columns to match the pattern in the dataset of [this huggingface dataset](https://huggingface.co/datasets/lmqg/qg_squad?row=0) proposed by the original paper.

In [ ]:
paragraph_doc = nlp(context)

paragraph_doc

Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro do edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços erguidos com a lenda "Venite Ad Me Omnes". Ao lado do edifício principal é a Basílica do Sagrado Coração. Imediatamente atrás da basílica é a Gruta, um lugar mariano de oração e reflexão. É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858. No final da unidade principal (e em uma linha direta que liga através de 3 estátuas e da Cúpula de Ouro), é um estátua de pedra simples e moderna de Maria.

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize, sent_tokenize


split_punct = lambda text_: re.sub(r'(\w+)(,|\.)(\)|\()', '\1 \2 \\3', text_)
new_context = split_punct(context)

matches = [re.search(f'{token}', new_context) for token in word_tokenize(answer)
                              if re.search(re.compile(f'{token}'), new_context)]
matches

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[<re.Match object; span=(540, 550), match='Bernadette'>,
 <re.Match object; span=(551, 560), match='Soubirous'>]

In [ ]:
def get_context_sentence(start, end, context_):

  start_sent = 0
  for i in range(start, 0, -1):
      if context_[i] == ".":
        start_sent = i + 2
        break
  end_sent = -1
  for i in range(end, len(context_)):
      if context_[i] == ".":
        end_sent = i+1
        break

  return start_sent, end_sent

In [ ]:
sent = sent_tokenize('Hoje eu vou para casa. Amanhã para a Igreja.')
sent

['Hoje eu vou para casa.', 'Amanhã para a Igreja.']

In [ ]:
def spacy_get_text_boundaries(text, context_doc):
    """
    Find the boundaries of the answers tokens in the context document
    text: string # answer in string format
    context_doc: spacy document # context where the answers boundaries must
                                #  be found
    """
    matcher = Matcher(nlp.vocab)

    pattern = [[{"ORTH": token.text} ]  for token in nlp(text)]
    matcher.add(text, pattern)
    matches = matcher(context_doc)
    print(text)
    print(context_doc)
    start_sent, end_sent = -1, -1

    answers_starts = []
    answers_ends = []
    for match_id, start, end in matches:
        answers_starts.append(start)
        answers_ends.append(end)

    return min(answers_starts), max(answers_ends)


In [ ]:
def get_text_boundaries(text_, context_):
    """
    Find the boundaries of the answers tokens in the context document
    text: string # answer in string format
    context_doc: spacy document # context where the answers boundaries must
                                #  be found
    """
    new_context = split_punct(context_)
    matches = [re.search(f'{token}', new_context) for token in word_tokenize(text_)
                              if re.search(f'{token}', new_context)]

    answers_starts = []
    answers_ends = []
    for match_patterm in matches:
        start, end = match_patterm.span()
        answers_starts.append(start)
        answers_ends.append(end)

    return min(answers_starts), max(answers_ends)


In [ ]:


start, end = spacy_get_text_boundaries(answer, paragraph_doc)
start, end

paragraph_doc[start]._.sent

Saint Bernadette Soubirous
Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro do edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços erguidos com a lenda "Venite Ad Me Omnes". Ao lado do edifício principal é a Basílica do Sagrado Coração. Imediatamente atrás da basílica é a Gruta, um lugar mariano de oração e reflexão. É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858. No final da unidade principal (e em uma linha direta que liga através de 3 estátuas e da Cúpula de Ouro), é um estátua de pedra simples e moderna de Maria.


É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858.

In [ ]:
from spacy.tokens import Doc

start, end =  get_text_boundaries(answer, context)

true_answer_text =  context[slice(*(start, end))]

get_sentence = lambda text_answer : [sent for sent in sent_tokenize(context)
                                         if re.search(re.compile(text_answer), sent)][0]
sent_text = get_sentence(true_answer_text)

true_answer_text, sent_text

('Bernadette Soubirous',
 'É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858.')

In [ ]:
start_sent, end_sent = get_context_sentence(start, end, context)

context[start_sent:end_sent]

'É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858.'

In [ ]:

def mark_text_span(text_, start_span, end_span,):
  text_list = list(text_)
  text_list.insert(start_span, "<h1>")
  text_list.insert(end_span+1, "<h1>")
  return ''.join(text_list)

In [ ]:
paragraph_answer = spacy_mark_text_span(paragraph_doc, start, end)
paragraph_answer

Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro do edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços erguidos com a lenda" Venite Ad Me Omnes". Ao lado do edifício principal é a Basílica do Sagrado Coração. Imediatamente atrás da basílica é a Gruta, um lugar mariano de oração e reflexão. É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858. No final da unidade principal( e em uma linha direta que liga através de 3 estátuas e da Cúpula de Ouro), é um estátua de pedra simples e moderna de Maria.<h1><h1>

In [ ]:
paragraph_answer = mark_text_span(context, start, end)
paragraph_answer

'Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro do edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços erguidos com a lenda "Venite Ad Me Omnes". Ao lado do edifício principal é a Basílica do Sagrado Coração. Imediatamente atrás da basílica é a Gruta, um lugar mariano de oração e reflexão. É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa <h1>Bernadette Soubirous<h1> em 1858. No final da unidade principal (e em uma linha direta que liga através de 3 estátuas e da Cúpula de Ouro), é um estátua de pedra simples e moderna de Maria.'

In [ ]:

sent_start, sent_end = re.search(sent_text, context).span()
paragraph_sentence = mark_text_span(context, sent_start, sent_end)
paragraph_sentence

'Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro do edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços erguidos com a lenda "Venite Ad Me Omnes". Ao lado do edifício principal é a Basílica do Sagrado Coração. Imediatamente atrás da basílica é a Gruta, um lugar mariano de oração e reflexão. <h1>É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858.<h1> No final da unidade principal (e em uma linha direta que liga através de 3 estátuas e da Cúpula de Ouro), é um estátua de pedra simples e moderna de Maria.'

In [ ]:

start_answer, end_answer =  get_text_boundaries(true_answer_text, sent_text)

sentence_answer = mark_text_span(sent_text, start_answer, end_answer)
sentence_answer

'É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa <h1>Bernadette Soubirous<h1> em 1858.'

## Todo

Criar uma função e usar map para alimentar o dataset com estas informações.

In [ ]:
def get_extra_info(example):

    context_text = example['paragraph']
    answer_text = example['answer']
    question = example['question']

    nlp_doc = nlp(context_text)

    start, end =  spacy_get_text_boundaries(answer_text, nlp_doc)
    true_answer_text =  nlp_doc[slice(*(start, end))]
    sentence = true_answer_text[0]._.sent
    sent_start, sent_end = sentence.start, sentence.end


    example['answer'] = true_answer_text.text
    example['sentence'] = sentence.text
    example['paragraph_answer'] = spacy_mark_text_span(nlp_doc, start, end).text
    example['paragraph_sentence'] = spacy_mark_text_span(nlp_doc, sent_start, sent_end).text

    start_answer, end_answer =  spacy_get_text_boundaries(answer_text, sentence)
    example['sentence_answer'] = spacy_mark_text_span(sentence, start_answer, end_answer).text
    return example


In [ ]:
final_dataset = processed_dataset['validation'].map(get_extra_info)

final_dataset

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Denver Broncos
Super Bowl 50 foi um jogo de futebol americano para determinar o campeão da National Football League (NFL) para a temporada de 2015. O campeão da American Football Conference (AFC), Denver Broncos, derrotou a campeã Carolina Panthers, da National Football Conference (NFC), por 24 a 10, e conquistou seu terceiro título no Super Bowl. O jogo foi disputado em 7 de fevereiro de 2016, no Levi&#39;s Stadium, na área da baía de San Francisco, em Santa Clara, Califórnia. Como este foi o 50º Super Bowl, a liga enfatizou o "aniversário de ouro" com várias iniciativas de ouro, bem como a suspensão temporária da tradição de nomear cada jogo do Super Bowl com algarismos romanos (sob os quais o jogo seria conhecido como " Super Bowl L "), para que o logotipo possa destacar os algarismos arábicos 50.
Denver Broncos
O campeão da American Football Conference (AFC), Denver Broncos, derrotou a campeã Carolina Panthers, da National Football Conference (NFC), por 24 a 10, e conquistou seu te

ValueError: ignored

In [ ]:
processed_dataset['validation']

In [ ]:
processed_dataset['validation'].select(range(7000,9000)).map(get_extra_info)